> `Annotator` is an `explorer` which provides a map of your data colored by labels.
>
> :speedboat: Let's walk through its components and how they interact with the `dataset`.
>
> -   {== You will find many of these components again in other `explorer`s. ==}

-   <details open><summary>This page addresses **single components** of `hover`</summary>
    We are using code snippets to pick out parts of the annotation interface, so that the documentation can explain what they do.

    -   Please be aware that this is NOT how one would typically use `hover`.
    -   Typical usage deals with [recipes](../../tutorial/t1-active-learning) where the individual parts have been tied together.

</details>

-   <details open><summary>Dependencies for {== local environments ==}</summary>
    When you run the code locally, you may need to install additional packages.

    To run the text embedding code on this page, you need:
```shell
    pip install spacy
    python -m spacy download en_core_web_md
```

    To render `bokeh` plots in Jupyter, you need:
```shell
    pip install jupyter_bokeh
```

    If you are using JupyterLab older than 3.0, use this instead ([reference](https://pypi.org/project/jupyter-bokeh/)):
```shell
    jupyter labextension install @jupyter-widgets/jupyterlab-manager
    jupyter labextension install @bokeh/jupyter_bokeh
```

</details>

## **Preparation**

As always, start with a ready-for-plot dataset:

In [1]:
from hover.core.dataset import SupervisableTextDataset
import pandas as pd

raw_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.5.0/20_newsgroups_raw.csv"
train_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.5.0/20_newsgroups_train.csv"

# for fast, low-memory demonstration purpose, sample the data
df_raw = pd.read_csv(raw_csv_path).sample(400)
df_raw["SUBSET"] = "raw"
df_train = pd.read_csv(train_csv_path).sample(400)
df_train["SUBSET"] = "train"
df_dev = pd.read_csv(train_csv_path).sample(100)
df_dev["SUBSET"] = "dev"
df_test = pd.read_csv(train_csv_path).sample(100)
df_test["SUBSET"] = "test"

# build overall dataframe and ensure feature type
df = pd.concat([df_raw, df_train, df_dev, df_test])
df["text"] = df["text"].astype(str)

# this class stores the dataset throught the labeling process
dataset = SupervisableTextDataset.from_pandas(df, feature_key="text", label_key="label")

🔵 SupervisableTextDataset: Initializing...

🔵 SupervisableTextDataset: Deduplicating...

🔵 SupervisableTextDataset: --subset raw rows: 400 -> 382.

🔵 SupervisableTextDataset: --subset train rows: 400 -> 382.

🔵 SupervisableTextDataset: --subset dev rows: 100 -> 97.

🔵 SupervisableTextDataset: --subset test rows: 100 -> 97.

🟢 SupervisableTextDataset: Set up label encoder/decoder with 20 classes.

🟢 SupervisableTextDataset: Population updater: latest population with 20 classes.

🟢 SupervisableTextDataset: SupervisableTextDataset: finished initialization.

<br>

In [2]:
import spacy
import re
from functools import lru_cache

# use your preferred embedding for the task
nlp = spacy.load("en_core_web_md")

# raw data (str in this case) -> np.array
@lru_cache(maxsize=int(1e+4))
def vectorizer(text):
    clean_text = re.sub(r"[\s]+", r" ", str(text))
    return nlp(clean_text, disable=nlp.pipe_names).vector

# any kwargs will be passed onto the corresponding reduction
# for umap: https://umap-learn.readthedocs.io/en/latest/parameters.html
# for ivis: https://bering-ivis.readthedocs.io/en/latest/api.html
reducer = dataset.compute_nd_embedding(vectorizer, "umap", dimension=2)

Vectorizing: 100%|██████████| 958/958 [00:02<00:00, 378.03it/s]


🔵 SupervisableTextDataset: Fit-transforming UMAP on 861 samples...

🔵 SupervisableTextDataset: Transforming UMAP on 97 samples...

🟢 SupervisableTextDataset: Computed 2-d embedding in columns ['embed_2d_0', 'embed_2d_1']

<br>

## **Scatter Plot: Semantically Similar Points are Close Together**

`hover` labels data points in bulk, which requires selecting groups of homogeneous data.

The core of the annotator is a scatter plot and labeling widgets:

In [3]:
from bokeh.io import show, output_notebook

output_notebook()

# normally your would skip notebook_url or use Jupyter address
notebook_url = 'localhost:8888'

from hover.recipes.subroutine import standard_annotator
from bokeh.layouts import row, column

annotator = standard_annotator(dataset)
show(column(
    row(annotator.annotator_input, annotator.annotator_apply),
    annotator.figure,
), notebook_url=notebook_url)

Loading BokehJS ...

🔵 BokehTextAnnotator: Setting up DataFrames

🔵 BokehTextAnnotator: Setting up sources

🔵 BokehTextAnnotator: Setting up widgets

🔵 BokehTextAnnotator: activated search base response on TextInput(id='1069', ...)

...

🟢 BokehTextAnnotator: Plotted subset train with 382 points

🟢 SupervisableTextDataset: Subscribed BokehTextAnnotator to dataset pushes: {'raw': 'raw', 'train': 'train', 
'dev': 'dev', 'test': 'test'}

🟢 SupervisableTextDataset: Subscribed BokehTextAnnotator to dataset commits: {'raw': 'raw'}

🟢 SupervisableTextDataset: Subscribed BokehTextAnnotator to selection table: ['raw', 'train', 'dev', 'test']

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<br>

### **Select Points on the Plot**

On the right of the scatter plot, you can find tap, polygon, and lasso tools which can select data points.

### **View Tooltips with Mouse Hover**

Embeddings are helpful but rarely perfect. This is why we have tooltips that show the detail of each point on mouse hover, allowing us to inspect points, discover patterns, and come up with new labels on the fly.

### **Show & Hide Subsets**

Showing labeled subsets can tell you which parts of the data has been explored and which ones have not. With toggle buttons, you can turn on/off the display for any subset.

In [4]:
show(annotator.data_key_button_group, notebook_url=notebook_url)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<br>

## **Make Consecutive Selections**

Ever selected multiple (non-adjacent) files in your file system using <kbd>Ctrl</kbd>/<kbd>Command</kbd>?

Similarly but more powerfully, you can make consecutive selections with a "keep selecting" option.

In [5]:
show(annotator.selection_option_box, notebook_url=notebook_url)

<br>

-   <details open><summary>Selection option values: what do they do?</summary>
    Basic set operations on your old & new selection. [Quick intro here](https://www.geeksforgeeks.org/python-set-operations-union-intersection-difference-symmetric-difference/)

    -   `none`: the default, where a new selection `B` simply replaces the old one `A`.
    -   `union`: `A ∪ B`, the new selection gets unioned with the old one.
        -   this resembles the <kbd>Ctrl</kbd>/<kbd>Command</kbd> mentioned above.
    -   `intersection`: `A ∩ B`, the new selection gets intersected with the old one.
        -   this is particularly useful when going beyond simple 2D plots.
    -   `difference`: `A ∖ B`, the new selection gets subtracted from the old one.
        -   this is for de-selecting outliers.
</details>

## **Change Plot Axes**

`hover` supports dynamically choosing which embedding dimensions to use for your 2D plot. This becomes nontrivial, and sometimes very useful, when we have a 3D embedding (or higher):

In [6]:
reducer = dataset.compute_nd_embedding(vectorizer, "umap", dimension=3)

annotator = standard_annotator(dataset)

show(column(
    row(annotator.dropdown_x_axis, annotator.dropdown_y_axis),
    annotator.figure,
), notebook_url=notebook_url)

Vectorizing: 100%|██████████| 958/958 [00:00<00:00, 1755414.26it/s]


🔵 SupervisableTextDataset: Fit-transforming UMAP on 861 samples...

🔵 SupervisableTextDataset: Transforming UMAP on 97 samples...

🟢 SupervisableTextDataset: Computed 3-d embedding in columns ['embed_3d_0', 'embed_3d_1', 'embed_3d_2']

🔵 BokehTextAnnotator: Setting up DataFrames

...

🟢 BokehTextAnnotator: Plotted subset train with 382 points

🟢 SupervisableTextDataset: Subscribed BokehTextAnnotator to dataset pushes: {'raw': 'raw', 'train': 'train', 
'dev': 'dev', 'test': 'test'}

🟢 SupervisableTextDataset: Subscribed BokehTextAnnotator to dataset commits: {'raw': 'raw'}

🟢 SupervisableTextDataset: Subscribed BokehTextAnnotator to selection table: ['raw', 'train', 'dev', 'test']

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<br>

## **Text Search Widget: Include/Exclude**

Keywords or regular expressions can be great starting points for identifying a cluster of similar points based on domain expertise.

You may specify a *positive* regular expression to look for and/or a *negative* one to not look for.

The `annotator` will amplify the sizes of positive-match data points and shrink those of negative matches.

In [7]:
show(row(annotator.search_pos, annotator.search_neg), notebook_url=notebook_url)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<br>

### **Preview: Use Search for Selection in Finder**

In a particular kind of plots called `finder` (see later in the tutorials), the search widget can directly operate on your selection as a filter.

## **The Plot and The Dataset**

When we apply labels through the annotator plot, it's acutally the `dataset` behind the plot that gets immediately updated. The plot itself is not in direct sync with the dataset, which is a design choice for performance. Instead, we will use a trigger called `PUSH` for updating the data entries to the plot.

### **PUSH: Synchronize from Dataset to Plots**

Below is the full interface of the `dataset`, where you can find a green "Push" button:

In [8]:
show(dataset.view(), notebook_url=notebook_url)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In a built-in `recipe`, the "Push" button will update the latest data to every `explorer` linked to the `dataset`.